In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import cvxpy

## Introduction to Portfolio Theory

In classical portfolio theory periodic returns are normally distributed. We will assume daily returns of two stocks $R_A$ and $R_B$ denote prices of two stocks $A$ and $B$, respectively, such that they are jointly normally distributed with the following properties. Marginally, 
$$R_A \sim N(\mu_A,\sigma_A^2) \text{, and } R_B \sim N(\mu_B,\sigma_B^2),$$
and, jointly, their covariance and correlation are
$$\sigma_\text{AB} = \text{Cov}(R_A,R_B) \text{, and } \rho_\text{AB} = \frac{\sigma_\text{AB}}{\sigma_A\sigma_B}$$

If we consider managing a portfolio consisting of these two stocks, then the return for the portfolio $R_p$ is 
$$R_p = R_A x_A + R_B x_B,$$
where $x_A$ and $x_B$ are fixed proportions of portfolio such that $x_A + x_B = 1$.

What are the properties of $R_p$? We know $R_p$ is also normally distributed with following mean and variance:
$$ 
\mu_p = \text{E}(R_p) = \text{E}(R_B)\, x_A + \text{E}(R_B)\, x_B = \mu_A x_A + \mu_B x_B\\
\sigma_p^2 = \text{Var}(R_p) = \text{E}\left((R_p - \text{E}(R_p))^2\right) = \sigma_A^2\,x_A^2 + \sigma_B^2\,x_B^2 + 2\sigma_{AB}x_A x_B.
$$
Note that $\sigma_{AB} = \rho_{AB}\sigma_A\sigma_B$.

Finally, the distribution of the portfolio return is
$$ R_p \sim N(\mu_p, \sigma_p^2) $$

Let's simulate behavior of returns $R_A$, $R_B$, and $R_p$ (portfolio weight distributed evenly): i.e. $x_A = x_B = 0.5$. Also, choose some parameters for our two stocks.

In [ ]:
x = pd.Series({'A': 0.5, 'B': 0.5})
mu = pd.Series({'A':0.15, 'B':0.1})
sig = pd.Series({'A':0.1, 'B':0.05})

## correlation between stocks
rhoAB = -0.15

### Calculate theoretical portfolio returns

Calculate the volatility ($\sigma^2$) and expected returns ($\mu$):

In [ ]:
mu['p'] = mu.A * x.A + mu.B * x.B
sig['p'] = np.sqrt((sig.A**2 * x.A**2) + (sig.B**2 * x.B**2) + (2 * rhoAB * sig.A *sig.B * x.A * x.B))

[one for one in zip(sig**2,mu)]

### Generate random returns

Generate random observations from bivariate normal distribution specified by 

In [ ]:
param_mu = [mu.A, mu.B]
param_sigma = [[sig.A**2, rhoAB*sig.A*sig.B],
               [rhoAB*sig.A*sig.B, sig.B**2]]

R_AB = np.random.multivariate_normal(param_mu, param_sigma, 1000)
RA = R_AB[:,0]
RB = R_AB[:,1]

print('RA: mean is', RA.mean(), ', std dev is', RA.std())
print('RB: mean is', RB.mean(), ', std dev is', RB.std())
print('Correlation between RA and RB is', np.corrcoef(RA, RB)[1,0])

pd.tools.plotting.scatter_matrix(pd.DataFrame(R_AB, columns=['RA', 'RB']));

Volatility of returns are:

In [ ]:
sig

Note that $\mu_p$ is a simple average of the returns; however, $\sigma_p^2 < 0.5\sigma_A^2 + 0.5\sigma_B^2$. This means that volatility is less than if $A$ and $B$ were independent. This owes to the characteristic that $\rho<0$, so that the cross-term subtracts from $\sigma_p^2$. In fact, this is theoretical basis for the advantage of diversification.

### Efficient frontier of portfolios

What happens if we change the proportion of the portfolios $x_A$ and $x_B$? For a range of possibilities, we compute the resulting return $\mu_p$ and volatility $\sigma_p^2$.

In [ ]:
xa = np.linspace(-1, 2, num=101)
pf = pd.DataFrame({'xa':xa, 'xb':1-xa})

pf['mu'] = mu.A * pf.xa + mu.B * pf.xb
pf['sig2'] = ((sig.A**2 * pf.xa**2) + (sig.B**2 * pf.xb**2) + (2 * rhoAB * sig.A *sig.B * pf.xa * pf.xb))

#### Long and short positions

Note that `xa` is allowed to be negative. A positive $x_A$ value indicates what is called a _long position_ for stock A and indicates a purchase. On the otherhand, a negative $x_A$ indicates a _short position_ stock A and indicates a sale. The sale of A allows for higher long positions on other stocks.

In [ ]:
p = plt.subplot(1,1,1)
plt.plot(pf.sig2, pf.mu, color='b')
plt.plot(pf.sig2[pf.xa<0], pf.mu[pf.xa<0], color='r')
plt.plot(pf.sig2[pf.xb<0], pf.mu[pf.xb<0], color='g')
plt.scatter(sig**2, mu)
p.set_xlabel('Portfolio volatility')
p.set_ylabel('Portfolio return')
plt.show(p)

Above plot is called the efficient frontier. The line represents the lower boundary of achievable volatility for each level of portfolio return using component stocks. Two stocks $A$ and $B$ are shown as dots in the interior of the parabola, and the dot on the parabola is the return and volatility of a portfolio in which $x_A = x_b = 0.5$.

#### Minimum variance portfolio (MVP)

What is the point at the tip of the parabola? This point indicates what is called the minimum variance portfolio that, regardless of the return, has the minimum volatility. Minimum variance portfolio (MVP) has the minimum volatility of any portfolio achievable.

It can be shown that minimum variance portfolio would have the following allocation $x_A$:

$$x_A = \frac{\sigma_B^2-\sigma_\text{AB}^2}{\sigma_A^2 + \sigma_B^2 – \sigma_\text{AB}^2}$$

In [ ]:
mvp_A = (sig.B**2 - rhoAB * sig.A *sig.B )/(sig.A**2 + sig.B**2 - rhoAB * sig.A *sig.B ) 
mvp_B = 1 - mvp_A
mvp_mu = mu.A * mvp_A + mu.B * mvp_B 
mvp_sig2 = ((sig.A**2 * mvp_A**2) + (sig.B**2 * mvp_B**2) + (2 * rhoAB * sig.A *sig.B * mvp_A * mvp_B))
mvp_mu, mvp_sig2

Plot the minimum variance portfolio:

In [ ]:
p = plt.subplot(1,1,1)
plt.plot(pf.sig2, pf.mu, color='b')
plt.plot(pf.sig2[pf.xa<0], pf.mu[pf.xa<0], color='r')
plt.plot(pf.sig2[pf.xb<0], pf.mu[pf.xb<0], color='g')
plt.scatter(mvp_sig2, mvp_mu, color='r', marker='+')
p.set_xlabel('Portfolio volatility')
p.set_ylabel('Portfolio return')
plt.show(p)

The mathematical problem to solve for the minimum variance portfolio can be stated as follows:
$$ \min_{x_A,x_B}\ \ \sigma_p^2 = \sigma_A^2 x_A^2 + \sigma_B^2 x_B^2 + \sigma_{AB} x_A x_B\\
\text{such that }x_A + x_B = 1$$
Here, $\sigma_p^2 = \sigma_A^2 x_A^2 + \sigma_B^2 x_B^2 + \sigma_{AB} x_A x_B$ is called the objective function, and $x_A + x_B = 1$ is called the constraint.

The constraint has to be satisfied exactly, and objective wants to be minimized with the freedom of changing the values of $x_A$ and $x_B$.

### Using `cvxpy` for numerical optimization

Calculating analytical solutions is not always possible. There are ways to compute solutions numerically. Although out of scope of our class, a large field in applied math called optimization. The tools from this area allow us to compute solutions for problems such as the one we have here. (For more information, visit the 

### Higher dimensional portfolios

In higher dimensions, the portfolio equantion is written in terms of vectors and matrices. For example, suppose we construct a portfolio consisting of $s$-assets. The portfolio allocation can be written as,
$$\mathbf{1}^\intercal x = 1,$$
where $x$ is an $s$-vector whose sum adds up to 1: i.e., $x_1 + x_2 + \cdots + x_s = 1$.

The portfolio volatility in matrix-vector form looks like:
$$
\begin{pmatrix}x_1 & x_2 & \cdots & x_s\end{pmatrix}
\begin{pmatrix}
\sigma_1^2 & \sigma_{12} & \cdots & \sigma_{1s}\\
\sigma_{21} & \sigma_2^2 & \cdots & \sigma_{2s}\\
\vdots & \vdots & & \vdots \\
\sigma_{s1} & \sigma_{s2} & \cdots & \sigma_s^2\\
\end{pmatrix}
\begin{pmatrix}x_1 \\ x_2 \\ \vdots \\ x_s\end{pmatrix}.
$$
To verify the two stock example result, we check $\sigma_p^2$:
$$
\sigma_p^2 = \begin{pmatrix}x_1 & x_2\end{pmatrix}
\begin{pmatrix}
\sigma_1^2 & \sigma_{12}\\
\sigma_{21} & \sigma_2^2\\
\end{pmatrix}
\begin{pmatrix}x_1 \\ x_2\end{pmatrix} = 
\begin{pmatrix}x_1 & x_2\end{pmatrix}
\begin{pmatrix}
\sigma_1^2 x_1 + \sigma_{12} x_2\\
\sigma_{21} x_1 + \sigma_2^2 x_2\\
\end{pmatrix} = 
\sigma_1^2 x_1^2 + \sigma_2^2 x_2^2 + \sigma_{12} x_1 x_2,
$$
which matches the result from before.

With this notation, we can re-write the problem.

$$ \min_{x\in\mathbb{R}^s}\ \ x^\intercal \Sigma x\\
\text{subject to }\mathbf{1}^\intercal x = 1,$$

It can be shown that the optimal solution to this problem is,
$$ x^* = (\mathbf{1}^\intercal\Sigma^{-1}\mathbf{1})^{-1}\Sigma^{-1}\mathbf{1} $$.

There are many variations of this problem. In fact the minimum variance portfolio is special case of a more general problem

$$ \min_{x\in\mathbb{R}^s}\ \ x^\intercal \Sigma x\\
\text{subject to }\mu^\intercal x\geq \mu^* \text{, and } \mathbf{1}^\intercal x = 1,$$

Note that in all of the above problems, the measure of $\mu^*$, expected returns, and covariance matrix have to be provided. In practice, these quantities are estimated from data.

### Download data from Quandl

Use the Quandl API to download some stock data. In this example, we will use the constituent stocks from the [Dow Jones Industrial Average](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average) to form our portfolio.

In [ ]:
# import mykeys as m
# import quandl
# 
# quandl.ApiConfig.api_key = None
# quandl.ApiConfig.api_key = m.apikeys['quandl']
# 
# # get the table for daily stock prices and,
# # filter the table for selected tickers, columns within a time range
# # set paginate to True because Quandl limits tables API to 10,000 rows per call
# 
# symbols = ['AAPL','AXP','BA','BAC','CAT',
#            'CSCO','CVX','DD','DWDP','DIS','GE',
#            'HD','HPQ','IBM','INTC','JNJ',
#            'JPM','KFT','KO','MCD','MMM',
#            'MRK','MSFT','PFE','PG','T',
#            'TRV','UTX','VZ','WMT','XOM']
# 
# data = quandl.get_table('WIKI/PRICES', ticker = symbols, 
#                         qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
#                         date = { 'gte': '2000-01-01', 'lte': '2018-05-01' }, 
#                         paginate=True)
# pickle.dump(data, open( "dowjones_data.pkl", "wb" ))
# 
# data.tail()

Since it is faster to load locally saved data, save the downloaded data into the file.

### Load saved data

In [ ]:
%%bash

wget https://www.quandl.com/api/v3/databases/WIKI/codes?api_key=DjK3WD8BVCXtBo6QfW-5
unzip -o codes\?api_key\=DjK3WD8BVCXtBo6QfW-5

In [ ]:
symbols = pd.read_csv('WIKI-datasets-codes.csv', header=None)
symbols.columns = ['code', 'description']
symbols['ticker'] = symbols.code.str.split('/', expand=True)[1]
symbols['description'] = symbols.description.str.split('\) Prices', expand=True)[1]
symbols.head()

In [ ]:
data = pickle.load(open("dowjones_data.pkl", "rb")).set_index('date')

Stock data can have irregularities such as missing data due stocks being added and removed from the index. Some examples are
* Alcoa Corp. (AA) was removed in 2013
* Apple (AAPL) was added in 2015
* E.I. du Pont de Nemours & Company (DD) was removed and replaced with Dow du Pont (DWDP) as a continuation in 2017

For simplicity the stocks we will use are based on the most recent DJIA constituent companies until 2017-08-31 so all 30 stocks can be included. Note that this is not the same as the Dow Jones Index throughout the time period since some symbols have entered the DJIA later than year 2000: e.g., Verizon was added in 2004.

We have set the index to be 'date' column. For the next line, unset datetime index in order to operate on the column:

In [ ]:
data_range = data.reset_index().groupby('ticker')['date'].agg([min, max])
data_range

#### Subsetting date ranges with datetime index
Setting `date` column as the index is useful for time series data. The row index can be used to subset the data:

In [ ]:
data = data['2000-01-03':'2017-08-31']
data = data.drop(['DWDP'])

## new data_range
data_range = data.reset_index().groupby('ticker')['date'].agg([min, max])
data_range

In [ ]:
data.head()

### Long to wide-format data

Since we will compute the covariance matrix, turn the data into long format

In [ ]:
datawide = data.reset_index().pivot(index='date',columns='ticker',values='adj_close')
datawide

### Log returns from stock prices

Stock market data are measured in terms of price per share. We need to compute the returns from the prices. 
Given the prices $P_t$ and $P_{t-1}$ where $t$ indicates time, the return at time $t$ is defined as
$$ R_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_t}{P_{t-1}} - 1 $$

It is well known that linear approximation of $\log(1+x)\approx x$ when $x$ is small. Since daily returns of stocks are small, we approximate that
$$ r_t = \log(1 + R_t) = \log\left(\frac{P_t}{P_{t-1}}\right) = \log(P_t) - \log(P_{t-1})$$
So, in order to compute the log-returns, compute the difference of log prices:

In [ ]:
logret = np.log(datawide).diff()
logret.head()

Note that the first time period is NaN since there is no data corresponding to $-1$.

Note that $100\cdot r_t$% represent daily percentage returns.

### Estimate expected returns

Estimate the daily expected returns by computing the means:

In [ ]:
mu = logret[1:].mean()
mu

### Estimate covariance matrix (volatility structure)

Estimate the covarince matrix of returns:

In [ ]:
sigma = logret.cov()
sigma

### Visualize volatility structure

We can visualize the relationship between the stocks by a heatmap.

In [ ]:
p = plt.figure(figsize=(18, 13))

ax = p.add_subplot(1,2,1)
plt.imshow(logret.cov())
plt.colorbar(orientation='horizontal')
ax.set_xticks(np.arange(0, 29, 1))
ax.set_yticks(np.arange(0, 29, 1))
ax.set_xticklabels(sigma.columns.values, rotation=90)
ax.set_yticklabels(sigma.columns.values)

ax = p.add_subplot(1,2,2)
plt.imshow(logret.corr())
plt.colorbar(orientation='horizontal')
ax.set_xticks(np.arange(0, 29, 1))
ax.set_yticks(np.arange(0, 29, 1))
ax.set_xticklabels(sigma.columns.values, rotation=90)
ax.set_yticklabels(sigma.columns.values)
plt.show(p)

In [ ]:
import cvxpy as cvx

s, _ = sigma.shape

w = cvx.Variable(s)
risk = cvx.quad_form(w, sigma.as_matrix())
prob = cvx.Problem(cvx.Minimize(risk), 
               [cvx.sum_entries(w) == 1])
prob.solve()
w.value

Examples for portfolio optimization: http://nbviewer.jupyter.org/github/cvxgrp/cvx_short_course/blob/master/applications/portfolio_optimization.ipynb

Even though a much more difficult problem to solve, we can use `cvxpy` package to solve optimization problems easily. Let's try to solve the more difficult problem with a target expected return:

$$ \min_{x\in\mathbb{R}^s}\ \ x^\intercal \Sigma x\\
\text{subject to }\mu^\intercal x\geq \mu^* \text{, and } \mathbf{1}^\intercal x = 1,$$

Note that in all of the above problems, the measure of $\mu^*$, expected returns, and covariance matrix have to be provided. In practice, these quantities are estimated from data.

In [ ]:
import cvxpy as cvx

s,_ = sigma.shape

w = cvx.Variable(s)
risk = cvx.quad_form(w, sigma.as_matrix())
prob = cvx.Problem(cvx.Minimize(risk), 
               [
                   cvx.sum_entries(w) == 1,
                   mu.as_matrix()*w >= 0.001
               ])
prob.solve()
w.value

Verify that the solution satisfies the constraints: i.e. $\mu^\intercal w=0.001$ and $\mathbf{1}^\intercal w=1$.

In [ ]:
w.value.sum()

In [ ]:
np.dot(mu.as_matrix(),w.value)

In [ ]:
earned = np.dot(logret.fillna(method='bfill').as_matrix(), w.value)
# f = plt.figure(figsize=(15,15))
# plt.plot((1+earned[1:]).cumprod())
# plt.show(f)

In [ ]:
earnedcp = (1+earned[1:]).cumprod()
ecp = pd.DataFrame(earnedcp.T, columns=['Portfolio Value'])
ecp.plot();

### Reality check

Obviously, this cannot be realistic. What aspects were unrealistic?